# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

We have designed the system as two M/M/1 queues put one after another. In the first queue, server is front desk opetor who we modeled as a resource in the SimPy environment. Similarly, expert is the server in the second queue who is also modeled as a resource in SimPy. 

Below, we first describe our code and later interpret the statistics and results we have gathered from the simulation.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
import itertools # for easy list operations
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_minutes):
    secs = int((total_minutes % 1)*60)
    mins = int(total_minutes % 60)
    hours = int(total_minutes //60)
    return f" {hours:02}:{mins:02}:{secs:02}"
        
# an object to hold expert queue statistics
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

# To run the simulation for 1000 customers, make sure that CUSTOMER_NUM equals CALLNUM1

In [3]:
# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

CUSTOMER_NUM=CALLNUM2

# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

LAST_EXIT = 0

LAST_CUSTOMER = 1

RANDOM_SEED = 978
random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []

# counter for breaks taken by the expert
break_count = 0



A said above, we modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        # it only action is happen, which can be thought as just being initiated
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue. \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue. \t at {format_time(self.finish_operator)}")
        

        with expert.request() as exp_req: 
                enter_e_q = self.env.now
                # customers may renege in the expert queue
                self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
                results = yield exp_req | env.timeout(self.patience)
                self.wait_exp=self.env.now-self.finish_operator
                second_queue_w_times.append(self.wait_exp) 
                exit_e_q = self.env.now
                global expert_queue_log
                expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

                if exp_req not in results:
                    #Leave queue, reneged
                    print(f"Cust {self.id:4} exited \t\t RENEGE, \t at {format_time(self.env.now)}")
                    print(f"-- after waiting {format_time(self.wait_exp)}")

                else:
                # We got to the expert
                    print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                    if self.wait_exp:
                        print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                    yield self.env.process(self.ask_question_expert())
                    self.finish_expert=self.env.now
                    print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        first_service_times.append(service_opr)
        yield self.env.timeout(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        second_service_times.append(service_exp)
        yield self.env.timeout(service_exp)

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [7]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        global LAST_CUSTOMER
        LAST_CUSTOMER = LAST_CUSTOMER + 1
 
        time_between_next_customer = random.expovariate(1/INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)

        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while LAST_CUSTOMER<CUSTOMER_NUM:
        time_to_break = random.expovariate(1/60)
 
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1

Here, we define our processes and resources and start running the simulation environment!

In [8]:

env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00
Cust    1 entered the 		 EXPERT Queue. 	 at  00:07:10
Cust    1 started talking with 	 EXPERT 	 at  00:07:10
Break is on the way 				 at  00:10:06
Cust    1 exited 		 NORMALLY 	 at  00:25:05
Break started 					 at  00:25:05
Break ended 					 at  00:28:05
Cust    2 entered the 		 FRONT Queue. 	 at  01:03:11
Cust    2 started talking with 	 FRONT Desk 	 at  01:03:11
Break is on the way 				 at  01:06:28
Break started 					 at  01:06:28
Cust    2 entered the 		 EXPERT Queue. 	 at  01:09:16
Break ended 					 at  01:09:28
Cust    2 started talking with 	 EXPERT 	 at  01:09:28
Cust    2 waited in 		 EXPERT Queue 	 for 00:00:12
Cust    3 entered the 		 FRONT Queue. 	 at  01:12:19
Cust    3 started talking with 	 FRONT Desk 	 at  01:12:19
Cust    3 entered the 		 EXPERT Queue. 	 at  01:18:57
Cust    4 entered the 		 FRONT Queue. 	 at  01:32:21
Cust    4 started talking with 	 FRONT D

Cust  363 waited in 		 EXPERT Queue 	 for 01:11:06
Cust  366 entered the 		 FRONT Queue. 	 at  88:22:24
Cust  366 started talking with 	 FRONT Desk 	 at  88:22:24
Cust  365 exited 		 RENEGE, 	 at  88:25:30
-- after waiting  01:01:34
Cust  366 entered the 		 EXPERT Queue. 	 at  88:26:52
Cust  363 exited 		 NORMALLY 	 at  88:27:03
Cust  364 started talking with 	 EXPERT 	 at  88:27:03
Cust  364 waited in 		 EXPERT Queue 	 for 01:12:34
Cust  364 exited 		 NORMALLY 	 at  88:38:09
Break started 					 at  88:38:09
Break ended 					 at  88:41:09
Cust  366 started talking with 	 EXPERT 	 at  88:41:09
Cust  366 waited in 		 EXPERT Queue 	 for 00:14:17
Cust  367 entered the 		 FRONT Queue. 	 at  88:43:56
Cust  367 started talking with 	 FRONT Desk 	 at  88:43:56
Cust  366 exited 		 NORMALLY 	 at  88:46:45
Cust  367 entered the 		 EXPERT Queue. 	 at  88:47:59
Cust  367 started talking with 	 EXPERT 	 at  88:47:59
Cust  367 exited 		 NORMALLY 	 at  88:52:12
Cust  368 entered the 		 FRONT Queue. 	 

Cust  666 started talking with 	 EXPERT 	 at  167:31:58
Cust  666 waited in 		 EXPERT Queue 	 for 00:01:31
Cust  666 exited 		 NORMALLY 	 at  167:32:15
Cust  667 entered the 		 EXPERT Queue. 	 at  167:38:25
Cust  667 started talking with 	 EXPERT 	 at  167:38:25
Cust  667 exited 		 NORMALLY 	 at  167:50:34
Cust  668 entered the 		 FRONT Queue. 	 at  168:11:25
Cust  668 started talking with 	 FRONT Desk 	 at  168:11:25
Break is on the way 				 at  168:18:16
Break started 					 at  168:18:16
Cust  668 entered the 		 EXPERT Queue. 	 at  168:18:45
Cust  669 entered the 		 FRONT Queue. 	 at  168:18:50
Cust  669 started talking with 	 FRONT Desk 	 at  168:18:50
Break ended 					 at  168:21:16
Cust  668 started talking with 	 EXPERT 	 at  168:21:16
Cust  668 waited in 		 EXPERT Queue 	 for 00:02:30
Break is on the way 				 at  168:22:07
Cust  669 entered the 		 EXPERT Queue. 	 at  168:25:28
Cust  670 entered the 		 FRONT Queue. 	 at  168:30:01
Cust  670 started talking with 	 FRONT Desk 	 at 

Cust 1035 entered the 		 FRONT Queue. 	 at  254:13:29
Cust 1033 entered the 		 EXPERT Queue. 	 at  254:18:06
Cust 1034 started talking with 	 FRONT Desk 	 at  254:18:06
Cust 1034 waited in 		 FRONT Queue 	 for 00:06:58
Cust 1033 started talking with 	 EXPERT 	 at  254:18:06
Cust 1034 entered the 		 EXPERT Queue. 	 at  254:24:14
Cust 1035 started talking with 	 FRONT Desk 	 at  254:24:14
Cust 1035 waited in 		 FRONT Queue 	 for 00:10:45
Cust 1036 entered the 		 FRONT Queue. 	 at  254:24:48
Cust 1033 exited 		 NORMALLY 	 at  254:27:19
Cust 1034 started talking with 	 EXPERT 	 at  254:27:19
Cust 1034 waited in 		 EXPERT Queue 	 for 00:03:04
Break is on the way 				 at  254:30:06
Cust 1034 exited 		 NORMALLY 	 at  254:31:04
Break started 					 at  254:31:04
Cust 1035 entered the 		 EXPERT Queue. 	 at  254:31:52
Cust 1036 started talking with 	 FRONT Desk 	 at  254:31:52
Cust 1036 waited in 		 FRONT Queue 	 for 00:07:03
Break ended 					 at  254:34:04
Cust 1035 started talking with 	 EXPERT

Cust 1465 exited 		 NORMALLY 	 at  359:11:43
Cust 1466 entered the 		 EXPERT Queue. 	 at  359:17:36
Cust 1466 started talking with 	 EXPERT 	 at  359:17:36
Break is on the way 				 at  359:20:25
Cust 1466 exited 		 NORMALLY 	 at  359:20:49
Break started 					 at  359:20:49
Break ended 					 at  359:23:49
Cust 1467 entered the 		 FRONT Queue. 	 at  359:41:27
Cust 1467 started talking with 	 FRONT Desk 	 at  359:41:27
Cust 1467 entered the 		 EXPERT Queue. 	 at  359:48:01
Cust 1467 started talking with 	 EXPERT 	 at  359:48:01
Cust 1468 entered the 		 FRONT Queue. 	 at  359:51:37
Cust 1468 started talking with 	 FRONT Desk 	 at  359:51:37
Cust 1468 entered the 		 EXPERT Queue. 	 at  359:59:09
Cust 1469 entered the 		 FRONT Queue. 	 at  360:09:20
Cust 1469 started talking with 	 FRONT Desk 	 at  360:09:20
Cust 1469 entered the 		 EXPERT Queue. 	 at  360:15:35
Cust 1467 exited 		 NORMALLY 	 at  360:16:19
Cust 1468 started talking with 	 EXPERT 	 at  360:16:19
Cust 1468 waited in 		 EXPERT 

...

-- after waiting  00:22:57
Cust 2977 entered the 		 EXPERT Queue. 	 at  721:16:11
Cust 2978 started talking with 	 FRONT Desk 	 at  721:16:11
Cust 2978 waited in 		 FRONT Queue 	 for 00:31:25
Cust 2985 entered the 		 FRONT Queue. 	 at  721:16:50
Cust 2986 entered the 		 FRONT Queue. 	 at  721:18:29
Cust 2972 exited 		 NORMALLY 	 at  721:19:00
Cust 2974 started talking with 	 EXPERT 	 at  721:19:00
Cust 2974 waited in 		 EXPERT Queue 	 for 00:23:00
Cust 2987 entered the 		 FRONT Queue. 	 at  721:21:06
Cust 2978 entered the 		 EXPERT Queue. 	 at  721:21:59
Cust 2979 started talking with 	 FRONT Desk 	 at  721:21:59
Cust 2979 waited in 		 FRONT Queue 	 for 00:34:57
Cust 2975 exited 		 RENEGE, 	 at  721:22:10
-- after waiting  00:18:14
Cust 2988 entered the 		 FRONT Queue. 	 at  721:23:24
Cust 2979 entered the 		 EXPERT Queue. 	 at  721:30:53
Cust 2980 started talking with 	 FRONT Desk 	 at  721:30:53
Cust 2980 waited in 		 FRONT Queue 	 for 00:42:59
Cust 2989 entered the 		 FRONT Queue. 	

Cust 3349 exited 		 RENEGE, 	 at  812:26:37
-- after waiting  00:07:22
Cust 3347 exited 		 NORMALLY 	 at  812:27:39
Break started 					 at  812:27:39
Break ended 					 at  812:30:39
Cust 3348 started talking with 	 EXPERT 	 at  812:30:39
Cust 3348 waited in 		 EXPERT Queue 	 for 00:18:46
Cust 3351 entered the 		 EXPERT Queue. 	 at  812:30:54
Cust 3352 started talking with 	 FRONT Desk 	 at  812:30:54
Cust 3352 waited in 		 FRONT Queue 	 for 00:11:19
Cust 3354 entered the 		 FRONT Queue. 	 at  812:32:00
Cust 3355 entered the 		 FRONT Queue. 	 at  812:32:09
Cust 3352 entered the 		 EXPERT Queue. 	 at  812:41:24
Cust 3353 started talking with 	 FRONT Desk 	 at  812:41:24
Cust 3353 waited in 		 FRONT Queue 	 for 00:19:12
Cust 3348 exited 		 NORMALLY 	 at  812:45:48
Break started 					 at  812:45:48
Break ended 					 at  812:48:48
Cust 3350 started talking with 	 EXPERT 	 at  812:48:48
Cust 3350 waited in 		 EXPERT Queue 	 for 00:22:53
Cust 3353 entered the 		 EXPERT Queue. 	 at  812:49:55


Cust 3794 started talking with 	 FRONT Desk 	 at  918:15:12
Cust 3794 waited in 		 FRONT Queue 	 for 00:08:03
Cust 3791 exited 		 NORMALLY 	 at  918:16:36
Cust 3792 started talking with 	 EXPERT 	 at  918:16:36
Cust 3792 waited in 		 EXPERT Queue 	 for 00:10:11
Cust 3794 entered the 		 EXPERT Queue. 	 at  918:23:44
Cust 3795 started talking with 	 FRONT Desk 	 at  918:23:44
Cust 3795 waited in 		 FRONT Queue 	 for 00:14:52
Cust 3792 exited 		 NORMALLY 	 at  918:28:41
Cust 3793 started talking with 	 EXPERT 	 at  918:28:41
Cust 3793 waited in 		 EXPERT Queue 	 for 00:13:28
Cust 3795 entered the 		 EXPERT Queue. 	 at  918:28:43
Cust 3796 entered the 		 FRONT Queue. 	 at  918:30:57
Cust 3796 started talking with 	 FRONT Desk 	 at  918:30:57
Cust 3796 entered the 		 EXPERT Queue. 	 at  918:37:46
Cust 3793 exited 		 NORMALLY 	 at  918:41:39
Cust 3794 started talking with 	 EXPERT 	 at  918:41:39
Cust 3794 waited in 		 EXPERT Queue 	 for 00:17:54
Cust 3794 exited 		 NORMALLY 	 at  918:54:56


Cust 4320 started talking with 	 FRONT Desk 	 at  1043:05:23
Cust 4320 waited in 		 FRONT Queue 	 for 00:08:08
Cust 4319 started talking with 	 EXPERT 	 at  1043:05:23
Cust 4319 exited 		 NORMALLY 	 at  1043:06:55
Cust 4320 entered the 		 EXPERT Queue. 	 at  1043:11:49
Cust 4321 started talking with 	 FRONT Desk 	 at  1043:11:49
Cust 4321 waited in 		 FRONT Queue 	 for 00:07:07
Cust 4320 started talking with 	 EXPERT 	 at  1043:11:49
Cust 4320 exited 		 NORMALLY 	 at  1043:16:08
Break is on the way 				 at  1043:16:18
Break started 					 at  1043:16:18
Break ended 					 at  1043:19:18
Cust 4321 entered the 		 EXPERT Queue. 	 at  1043:19:36
Cust 4321 started talking with 	 EXPERT 	 at  1043:19:36
Cust 4321 exited 		 NORMALLY 	 at  1043:29:15
Break is on the way 				 at  1043:42:10
Break started 					 at  1043:42:10
Cust 4322 entered the 		 FRONT Queue. 	 at  1043:43:17
Cust 4322 started talking with 	 FRONT Desk 	 at  1043:43:17
Break ended 					 at  1043:45:10
Cust 4322 entered the 		 

Cust 4763 started talking with 	 EXPERT 	 at  1143:25:37
Cust 4764 entered the 		 EXPERT Queue. 	 at  1143:33:46
Cust 4765 entered the 		 FRONT Queue. 	 at  1143:35:36
Cust 4765 started talking with 	 FRONT Desk 	 at  1143:35:36
Cust 4766 entered the 		 FRONT Queue. 	 at  1143:40:57
Cust 4763 exited 		 NORMALLY 	 at  1143:41:06
Cust 4764 started talking with 	 EXPERT 	 at  1143:41:06
Cust 4764 waited in 		 EXPERT Queue 	 for 00:07:19
Cust 4765 entered the 		 EXPERT Queue. 	 at  1143:42:48
Cust 4766 started talking with 	 FRONT Desk 	 at  1143:42:48
Cust 4766 waited in 		 FRONT Queue 	 for 00:01:50
Cust 4767 entered the 		 FRONT Queue. 	 at  1143:47:22
Cust 4764 exited 		 NORMALLY 	 at  1143:48:04
Cust 4765 started talking with 	 EXPERT 	 at  1143:48:04
Cust 4765 waited in 		 EXPERT Queue 	 for 00:05:16
Cust 4765 exited 		 NORMALLY 	 at  1143:50:17
Cust 4766 entered the 		 EXPERT Queue. 	 at  1143:51:58
Cust 4767 started talking with 	 FRONT Desk 	 at  1143:51:58
Cust 4767 waited in 		 

Lastly, we print our statistics we gathered from the simulation

In [9]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

# this makes sure everything went smoothly 
assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys if sys else 0 for w, sys in zip(total_waits, total_system)))

# a helper function to calculate avg length of expert queue
def log_to_avg_len(log):
   ins = sorted((entry.enter_e_q for entry in log))
   outs = sorted((entry.exit_e_q for entry in log))
   significant_points = np.array(sorted(ins + outs))
   in_mask = np.array([i in ins for i in significant_points])
   out_mask = np.array([o in outs for o in significant_points])
   queue_len = np.zeros(len(significant_points))

   
   for i, is_in in enumerate(in_mask):
      if is_in:
         queue_len[i:] += 1

   for i, is_in in enumerate(out_mask):
      if is_in:
         queue_len[i:] -= 1

   timedeltas = np.copy(significant_points)
   for i, elem in enumerate(timedeltas):
      timedeltas[i+1:] -= elem

   return np.sum( timedeltas*queue_len ) / outs[-1]

stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": log_to_avg_len(expert_queue_log),
}

#print("Number of breaks taken: %d" % (stats["break_count"]))
print("Utilization of front desk operator: %f%%" % (stats["util_front"]))
print("Utilization of expert operator: %f%%" % (stats["util_exp"]))
print("Average total waiting time: %f" % (stats["avg_total_wait"]))
print("Maximum of total waiting time to system time ratios: %f%%" % (stats["max_wait_2_sys_ratio"]))
print("Average length of expert waiting queue: %f" % (stats["avg_len_of_exp_q"]))


Utilization of front desk operator: 0.496577%
Utilization of expert operator: 0.607848%
Average total waiting time: 12.786866
Maximum of total waiting time to system time ratios: 0.931504%
Average length of expert waiting queue: 0.746078


# Statistics of 1000 customers

* Utilization of front desk operator: 0.487187%
* Utilization of expert operator: 0.597796%
* Average total waiting time: 11.670236
* Maximum of total waiting time to system time ratios: 0.894770%
* Average length of expert waiting queue: 0.687378


# Statistics of 5000 customers

* Utilization of front desk operator: 0.496577%
* Utilization of expert operator: 0.607848%
* Average total waiting time: 12.786866
* Maximum of total waiting time to system time ratios: 0.931504%
* Average length of expert waiting queue: 0.746078


From the above statistics, we can deduce following conclusions:

* Averagely half of the time, front desk operator sits idle. Since we have only one server, which is the operator, in this case, we can't reduce the number of servers to increase utilization. However, we can conclude that system doesn't look to be overloaded.

* Utilization for expert operator is a little higher than 50%, we can conclude that expert operator works within reasonable service boundaries.

* Average total waiting time is almost 13 minutes, which is actually not quite acceptable. However, interrarrival times are 14 minutes. So, waiting time looks to be acceptable in this case.

* Maximum waiting time to system time ratio is quite high, almost 94% percent. This means some customer(s) spend(s) their time in system almost only waiting. This is not acceptable in the customers perspective. This statistic shows that when there are an abundance of customers, system may have some queues.

* Average length of expert queue is quite less. However, other statistics are more meaningful.

So, we can conclude that the system is not overloaded but may have unacceptable waiting times in cases of abundance of customers. Administrators may consider increasing number of servers to remove this problem, if cost of an extra server is not too high.

In [10]:
print(format_time(env.now))
print(format_time(LAST_EXIT))

 1201:17:24
 1198:23:59
